<a href="https://colab.research.google.com/github/GoTudering/Final_Project_MTH3033/blob/main/SAlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
from glob import glob

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NUM_EPOCHS = 30
BATCH_SIZE = 128

CHECKPOINTS_DIR = './models.1'
os.makedirs(CHECKPOINTS_DIR, exist_ok=True)

In [3]:
train_set = datasets.FashionMNIST(root='FashionMNIST_data/',
                                  train=True,
                                  transform=transforms.ToTensor(),
                                  download=True)
test_set = datasets.FashionMNIST(root='FashionMNIST_data/',
                                 train=False,
                                 transform=transforms.ToTensor(),
                                 download=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting FashionMNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST_data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting FashionMNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST_data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting FashionMNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST_data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting FashionMNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST_data/FashionMNIST/raw



In [7]:
class SAlexNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        num_classes: int = 1000
        dropout: float = 0.5

        # Input: (N, 1, 28, 28)
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=6, stride=2, padding=0),  # (N, 12, 12, 16)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=5, stride=1, padding=0),  # (N, 8, 8, 16)
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),  # (N, 8, 8, 32)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=4, stride=1, padding=0),  # (N, 5, 5, 32)
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # (N, 5, 5, 64)
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # (N, 5, 5, 64)
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),  # (N, 5, 5, 32)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),  # (N, 2, 2, 32)
        ) # Output: (N, 32)

        # Input: (N, 32)
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(32 * 2 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        ) # Output: (N, 1000)

    def _initialize(self):
        for layer in self.features:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weigh, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        for i in (3, 8, 10):
            nn.init.constant_(self.features[i].bias, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = x.view(-1, 2 * 2 * 32)
        x = self.classifier(x)
        return x

In [8]:
seed = torch.initial_seed()
print(f"Seed: {seed}")

model = SAlexNet()
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
train_loader = data.DataLoader(train_set, shuffle=True,
                               pin_memory=True, num_workers=2,
                               drop_last=True, batch_size=BATCH_SIZE)

print("Start training...")
total_steps = 1
for epoch in range(NUM_EPOCHS):
    pbar = tqdm(train_loader, ascii=True, ncols=80)
    loss_records = []
    for imgs, classes in pbar:
        imgs, classes = imgs.to(device), classes.to(device)

        # Calculate the loss
        output = model(imgs)
        loss = F.cross_entropy(output, classes)

        # Update the parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_records.append(loss.item())

        # Log
#         if total_steps % 50 == 0:
#             with torch.no_grad():
#                 model.eval()
#                 _, preds = torch.max(output, 1)
#                 accuracy = torch.sum(preds == classes)

#                 print(f"Epoch: {epoch + 1} \tStep: {total_steps} \tLoss: {loss.item():.4f} \tAcc: {accuracy.item()}/{BATCH_SIZE}")
#                 model.train()
        total_steps += 1
    loss_mean = np.mean(loss_records)
    print(f"Epoch: {epoch + 1} \tStep: {total_steps} \tLoss: {loss_mean:.4f}")
    lr_scheduler.step()

    checkpoint_path = os.path.join(CHECKPOINTS_DIR, f"alexnet_states_e{epoch + 1:04d}.pkl")
    state = {
        'epoch': epoch,
        'total_steps': total_steps,
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'seed': seed,
    }
    torch.save(state, checkpoint_path)

Seed: 9722867967758336569
Start training...


100%|#########################################| 468/468 [00:53<00:00,  8.71it/s]


Epoch: 1 	Step: 469 	Loss: 1.6550


100%|#########################################| 468/468 [00:59<00:00,  7.85it/s]


Epoch: 2 	Step: 937 	Loss: 0.6087


...

Epoch: 28 	Step: 13105 	Loss: 0.2303


100%|#########################################| 468/468 [01:16<00:00,  6.09it/s]


Epoch: 29 	Step: 13573 	Loss: 0.2292


100%|#########################################| 468/468 [01:15<00:00,  6.21it/s]


Epoch: 30 	Step: 14041 	Loss: 0.2267


In [9]:
test_loader = data.DataLoader(test_set, shuffle=True,
                              pin_memory=True, num_workers=2,
                              batch_size=BATCH_SIZE)
model_paths = glob(os.path.join(CHECKPOINTS_DIR, 'alexnet_states_e????.pkl'))
model_paths.sort()

accuracy_records = []

for model_path in model_paths:
    chk = torch.load(model_path)
    epoch = chk['epoch']
    
    model = SAlexNet()
    model.load_state_dict(chk['model'])  # Load parameters from the checkpoint
    model = model.to(device)
  
    correct_coutner = 0
    pbar = tqdm(test_loader, ascii=True, ncols=80, total=len(test_set))
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)
        
        with torch.no_grad():
            output = model(imgs)
            model.eval()
            _, preds = torch.max(output, 1)
            accuracy = torch.sum(preds == labels)
            correct_coutner += accuracy.item()
        pbar.update(BATCH_SIZE)

    print(f"Epoch: {epoch + 1} \tAcc: {correct_coutner}/{len(test_set)}")
    accuracy_records.append(dict(epoch=(epoch + 1), correct = correct_coutner, path=model_path))

accuracy_records.sort(reverse=True, key=lambda x: x['correct'])
print(accuracy_records[0])    

  1%|3                                       | 79/10000 [00:03<07:55, 20.85it/s]


Epoch: 1 	Acc: 7553/10000


  1%|3                                       | 79/10000 [00:03<07:50, 21.10it/s]


Epoch: 2 	Acc: 8032/10000


  1%|3                                       | 79/10000 [00:03<07:37, 21.66it/s]


...

Epoch: 28 	Acc: 8794/10000


  1%|3                                       | 79/10000 [00:03<07:39, 21.60it/s]


Epoch: 29 	Acc: 8830/10000


  1%|3                                       | 79/10000 [00:03<07:43, 21.42it/s]

Epoch: 30 	Acc: 8870/10000
{'epoch': 30, 'correct': 8870, 'path': './models.1/alexnet_states_e0030.pkl'}
